In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Rheumatoid_Arthritis"
cohort = "GSE97475"

# Input paths
in_trait_dir = "../../input/GEO/Rheumatoid_Arthritis"
in_cohort_dir = "../../input/GEO/Rheumatoid_Arthritis/GSE97475"

# Output paths
out_data_file = "../../output/preprocess/Rheumatoid_Arthritis/GSE97475.csv"
out_gene_data_file = "../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE97475.csv"
out_clinical_data_file = "../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE97475.csv"
json_path = "../../output/preprocess/Rheumatoid_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"RA-MAP: mapping molecular immunological landscapes in early rheumatoid arthritis and healthy vaccine recipients [Healthy Hepatitis B Vaccine Recipients]"
!Series_summary	"Rheumatoid arthritis (RA) is a chronic inflammatory disorder with poorly defined aetiology characterised by synovial inflammation with variable disease severity and drug responsiveness. To investigate the peripheral blood immune cell landscape of RA, we performed comprehensive clinical and molecular profiling of 267 RA patients and 52 vaccine recipient controls for up to 18 months to establish a high quality sample biobank including plasma, serum, peripheral blood cells, urine, genomic DNA, RNA from whole blood, lymphocyte and monocyte subsets. We have performed extensive multi-omic immune phenotyping, including genomic, metabolomic, proteomic, transcriptomic and autoantibody profiling. We anticipate that these detailed clinical and molecular data will serve as a fundamental reso

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this is likely to contain gene expression data
# The title mentions "mapping molecular immunological landscapes" and the summary mentions "transcriptomic" profiling
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Finding trait row: This is a healthy control group for Rheumatoid Arthritis study,
# so all subjects are healthy. There's no explicit "disease status" field, but we can
# infer this from the title (these are healthy vaccine recipients).
trait_row = None  # No explicit trait data, but we can infer it

# Finding age row: Key 81 contains age data
age_row = 81

# Finding gender row: Key 118 contains gender data
gender_row = 118

# 2.2 Data Type Conversion Functions
# For trait (Rheumatoid Arthritis), we need to create this since it's not directly in the data
def convert_trait(value):
    # Since this is explicitly a "Healthy Hepatitis B Vaccine Recipients" dataset according to the title,
    # everyone is healthy (value = 0 for control/no disease)
    return 0

# For age, convert to numeric value
def convert_age(value):
    # Extract age value after colon and convert to integer
    try:
        if ':' in value:
            age_str = value.split(': ')[1].strip()
            return int(age_str)
        return None
    except:
        return None

# For gender, convert to binary (0 for female, 1 for male)
def convert_gender(value):
    # Extract gender value after colon
    if ':' in value:
        gender = value.split(': ')[1].strip()
        if gender.lower() == 'female':
            return 0
        elif gender.lower() == 'male':
            return 1
    return None

# 3. Save Metadata (Initial Filtering)
# Since we're creating trait data from a known dataset context (all healthy), we can mark trait as available
is_trait_available = True
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if is_trait_available:
    # First extract age and gender data if available
    import pandas as pd
    
    feature_list = []
    
    # Get patient IDs from the dataset
    patient_ids = set()
    for value in sample_characteristics_dict.get(1, []):
        if value.startswith('patientid:'):
            patient_id = value.split(': ')[1].strip()
            patient_ids.add(patient_id)
    
    # Create a basic DataFrame with sample identifiers
    clinical_data = pd.DataFrame({
        'ID_REF': list(patient_ids)
    })
    
    # For samples where we have age data
    if age_row is not None:
        # Create a temporary DataFrame to hold sample IDs and age values
        temp_age_data = {}
        for i, value in enumerate(sample_characteristics_dict.get(age_row, [])):
            if i < len(patient_ids):
                age_value = convert_age(value)
                if age_value is not None:
                    temp_age_data[list(patient_ids)[i]] = age_value
        
        if temp_age_data:
            age_df = pd.DataFrame({
                'ID_REF': list(temp_age_data.keys()),
                'Age': list(temp_age_data.values())
            })
            feature_list.append(age_df)
    
    # For samples where we have gender data
    if gender_row is not None:
        # Create a temporary DataFrame to hold sample IDs and gender values
        temp_gender_data = {}
        for i, value in enumerate(sample_characteristics_dict.get(gender_row, [])):
            if i < len(patient_ids):
                gender_value = convert_gender(value)
                if gender_value is not None:
                    temp_gender_data[list(patient_ids)[i]] = gender_value
        
        if temp_gender_data:
            gender_df = pd.DataFrame({
                'ID_REF': list(temp_gender_data.keys()),
                'Gender': list(temp_gender_data.values())
            })
            feature_list.append(gender_df)
    
    # Create a trait DataFrame with all subjects having trait=0 (healthy controls)
    trait_df = pd.DataFrame({
        'ID_REF': list(patient_ids),
        trait: [0] * len(patient_ids)  # All are healthy controls
    })
    feature_list.append(trait_df)
    
    # Merge all features
    from functools import reduce
    if len(feature_list) > 1:
        selected_clinical_df = reduce(lambda left, right: pd.merge(left, right, on='ID_REF', how='outer'), feature_list)
    else:
        selected_clinical_df = feature_list[0]
    
    # Preview the resulting dataframe
    preview = preview_df(selected_clinical_df)
    print("Clinical Data Preview:", preview)
    
    # Save to CSV
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)


Clinical Data Preview: {'ID_REF': ['0AB27C82FA2B25A2E3A7980D6B835980A3AE7C0AE79A8D081BCCD00A8337E173', '1C6309152C244CB158E689F773AD469F35C9276745A5250EBBB44F6EC2649F4A', '207463A32B6C5719066818B12A9BAF9C3E58982DFC878A09E4A7D3B952138E1E', '237CEB8CDA21A4B2EFB3B4B8920F13CCFEA42150B56FD24ABB8190F12F1E2BEE', '26365906D406AF22FCAABEF719F246AF2B639D477F942DA36D4C4E88942E8F79'], 'Age': [23.0, 45.0, nan, 21.0, nan], 'Gender': [nan, nan, nan, nan, nan], 'Rheumatoid_Arthritis': [0, 0, 0, 0, 0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['A1BG', 'A26C3', 'A2LD1', 'A4GNT', 'AAAS', 'AACS', 'AACSL', 'AADACL1',
       'AADAT', 'AAGAB', 'AAK1', 'AAMP', 'AARS', 'AARS2', 'AARSD1', 'AASDH',
       'AASDHPPT', 'AASS', 'AATF', 'AATK'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Based on the output, I can see that the gene identifiers in the gene expression data 
# appear to be standard human gene symbols/names (like A1BG, AAAS, AACS, etc.)
# These are official HGNC (HUGO Gene Nomenclature Committee) gene symbols
# and do not require mapping to other identifiers.

# No gene mapping is required since we already have standard gene symbols
requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Manual creation of clinical features when trait_row is None
sample_columns = clinical_data.columns.drop('!Sample_geo_accession') if '!Sample_geo_accession' in clinical_data.columns else clinical_data.columns
clinical_features = pd.DataFrame(index=[trait, 'Age', 'Gender'])

# Add constant trait values (all healthy)
clinical_features[sample_columns] = None  # Initialize with None
clinical_features.loc[trait, :] = 0  # All are healthy controls

# Add age data if available
if age_row is not None:
    for i, col in enumerate(sample_columns):
        if i < len(sample_characteristics_dict.get(age_row, [])):
            clinical_features.loc['Age', col] = convert_age(sample_characteristics_dict[age_row][i])

# Add gender data if available
if gender_row is not None:
    for i, col in enumerate(sample_columns):
        if i < len(sample_characteristics_dict.get(gender_row, [])):
            clinical_features.loc['Gender', col] = convert_gender(sample_characteristics_dict[gender_row][i])

selected_clinical_data = clinical_features

print("Clinical data preview:")
print(preview_df(selected_clinical_data))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_data.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 2. Normalize the gene symbols and save the gene data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_data, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview:")
print(preview_df(linked_data))

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and some demographic features are severely biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="All subjects are healthy controls (Hepatitis B vaccine recipients) with no disease status variation"
)

# 7. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Usable linked data saved to {out_data_file}")
else:
    print("Linked data was not usable and was not saved.")

Clinical data preview:
{'ID_REF': [0, 60, 1]}
Clinical data saved to ../../output/preprocess/Rheumatoid_Arthritis/clinical_data/GSE97475.csv


Normalized gene data saved to ../../output/preprocess/Rheumatoid_Arthritis/gene_data/GSE97475.csv
Linked data shape: (159, 13922)
Linked data preview:


{'Rheumatoid_Arthritis': [0, nan, nan, nan, nan], 'Age': [60, nan, nan, nan, nan], 'Gender': [1, nan, nan, nan, nan], 'A1BG': [nan, 5.082479526, 4.671344035, 5.441449056, 4.712100566], 'A4GNT': [nan, 4.661940564, 4.81966453, 4.938585051, 5.049440835], 'AAAS': [nan, 5.429443172, 5.371136621, 5.559627136, 5.350362338], 'AACS': [nan, 5.441156004, 5.371544678, 5.039645535, 5.165695714], 'AACSP1': [nan, 4.674130806, 5.537837503, 5.645531548, 5.528588519], 'AADAT': [nan, 4.681181511, 4.978846822, 4.873748716, 4.964660218], 'AAGAB': [nan, 8.060305984, 7.352844803, 7.585739382, 7.732203703], 'AAK1': [nan, 5.87366947, 8.391921638, 7.350749653, 7.481426519], 'AAMDC': [nan, 8.0920738, 7.076411543, 6.692440399, 7.065875255], 'AAMP': [nan, 6.940348678, 6.607801254, 6.685860341, 6.674887625], 'AAR2': [nan, 8.405592602, 8.593051918, 8.930993396, 8.801664223], 'AARS1': [nan, 9.886154661, 10.14403429, 10.42154106, 10.28189817], 'AARS2': [nan, 6.601856041, 6.86495815, 6.795407718, 7.145466307], 'AARSD1'